In [ ]:
import kagglehub

# Download the latest version of the dataset
path = kagglehub.dataset_download("manjilkarki/deepfake-and-real-images")

print("Path to dataset files:", path)

100%|██████████| 1.68G/1.68G [01:25<00:00, 21.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/manjilkarki/deepfake-and-real-images/versions/1


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os

# Define the base path to the dataset
base_path = '/root/.cache/kagglehub/datasets/manjilkarki/deepfake-and-real-images/versions/1/Dataset'

# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define paths to train, validation, and test sets
train_dir = os.path.join(base_path, 'Train')
val_dir = os.path.join(base_path, 'Validation')
test_dir = os.path.join(base_path, 'Test')

# Create datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the pre-trained EfficientNet model
model = models.efficientnet_b0(pretrained=True)

# Modify the classifier layer to match the number of output classes (real or fake)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

# Validation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation Accuracy: {100 * correct / total:.2f}%')

# Test the model
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 171MB/s]


Epoch [1/10], Loss: 0.0747
Epoch [2/10], Loss: 0.0483
Epoch [3/10], Loss: 0.0395
Epoch [4/10], Loss: 0.0340
Epoch [5/10], Loss: 0.0292
Epoch [6/10], Loss: 0.0263
Epoch [7/10], Loss: 0.0239
Epoch [8/10], Loss: 0.0218
Epoch [9/10], Loss: 0.0206
Epoch [10/10], Loss: 0.0181
Validation Accuracy: 98.11%
Test Accuracy: 91.10%


In [ ]:
# Define the file path in Google Drive to save the model weights
model_save_path = '/content/drive/My Drive/deepfake_detection_model_sahil.pth'

# Save the model's state dictionary
torch.save(model.state_dict(), model_save_path)

print(f'Model weights saved to {model_save_path}')


Model weights saved to /content/drive/My Drive/deepfake_detection_model_sahil.pth


In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# Load the model architecture
model = models.efficientnet_b0(pretrained=False)  # Don't load pretrained weights
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)

# Load the saved state dictionary
model.load_state_dict(torch.load('/content/drive/My Drive/deepfake_detection_model.pth'))

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print('Model weights loaded successfully')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-7-0ba1b8c9fd83>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommen

Model weights loaded successfully


In [ ]:
from PIL import Image
from torchvision import transforms

def predict_image(image_path):
    # Load the image
    image = Image.open(image_path).convert('RGB')

    # Define the same transformations used during training
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Transform the image
    image = transform(image).unsqueeze(0)  # Add a batch dimension

    # Move the image to the appropriate device
    image = image.to(device)

    # Set the model to evaluation mode
    model.eval()

    # Make the prediction
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)

    # Map predicted label to class name
    class_names = ['real', 'fake']
    return class_names[predicted.item()]

# Example usage
image_path = '/content/real_10002.jpg'  # Replace with your image path
result = predict_image(image_path)
print(f'The image is: {result}')


The image is: fake
